In [66]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
from tqdm.notebook import tqdm
import pandas as pd

In [114]:
url = 'https://sauce.foodpolis.kr/home/specialty/foodDbSearch.do?PAGE_MN_ID=SIS-030101'
header = {'User-Agent':'Mozila/5.0'}
ingre = requests.get(url, headers=header).text

In [115]:
ingre_soup = BeautifulSoup(ingre, 'lxml')
print(ingre_soup)

<!DOCTYPE html>
<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<title>식재료 DB</title>
<link href="/css/new2021/default.css" rel="stylesheet" type="text/css"/>
<link href="/css/new2021/layout.css" rel="stylesheet" type="text/css"/>
<link href="/css/new2021/sub.css" rel="stylesheet" type="text/css"/>
<link href="/css/new2021/content.css" rel="stylesheet" type="text/css"/>
<link href="/css/new2021/board.css" rel="stylesheet" type="text/css"/>
<link href="/css/new2021/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="/js/new2021/css/swiper.min.css" rel="stylesheet" type="text/css"/>
<link href="/js/new2021/css/slick.css" rel="stylesheet" type="text/css"/>
<link href="/css/new2021/owl.css" rel="stylesheet" type="text/css"/>
<script src="/js/new2021/jquery-3.3.1.min.js" type="text/javascript"></script>
<sc

In [129]:
t_list = ingre_soup.select('td')
t_list[2].get_text()

'Katsuo(일명), BONITO'

In [24]:
a_list = ingre_soup.select('div.modal-header')
b_list = ingre_soup.select('div.modal-body')
print(len(a_list),len(b_list))
print(a_list[0])

12 12
<div class="modal-header">
<h4 class="modal-title" id="myModalLabel">가다랑어</h4>
<button aria-label="Close" class="close" data-dismiss="modal" type="button"><span aria-hidden="true">×</span></button>
</div>


In [65]:
a_list[0].get_text()[1:-3]

'가다랑어'

In [102]:
b_list[0].get_text()

'\n\n\n\n재료 설명\n선형 녹색 잎은 온화한 양파 맛을 가지고 있으며, 생으로 먹거나 잘게 다져서 향채대신 이용한다.\n\n\n재료 특징\n\n\n\n이명\n알리움 암페로프라숨\n\n\n학명\nThinopyrum intermedium, Agropyron intermedium, Elytrigia intermedia, Elymus hispidus, Agropyron glaucum\n\n\n'

In [73]:
b_list[1].get_text().split("이명")[1].split("학명")[0][1:-3]

'가츠오부시'

In [35]:
titles=[]
for elem in a_list:
    titles.append(elem.get_text()[1:-3])
    
print(len(titles))
print(titles)

12
['가다랑어', '가다랑어포', '가리비', '가리비살', '가시복', '가자미', '가지', '가지열매', '가츠오부시', '간고등어', '간장', '갈매기살']


In [7]:
driver = webdriver.Chrome("./driver/chromedriver")
driver.get('https://sauce.foodpolis.kr/home/specialty/foodDbSearch.do?PAGE_MN_ID=SIS-030101')
nextpage = driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/div[4]/ul/li[4]/a')
nextpage.click()

# //*[@id="content"]/div[2]/div[4]/ul/li[3]/a
# //*[@id="content"]/div[2]/div[4]/ul/li[8]/a

C:\Users\BIT\AppData\Local\Temp\ipykernel_8176\1327422125.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver")


In [90]:
tmp_dict = ({'name':[],'diff':[]})
tmp_data = pd.DataFrame(tmp_dict)
tmp_data.loc[1]=["간고등어","간한 고등어"]

tmp_data

,name,diff
1,간고등어,간한 고등어


In [132]:
driver = webdriver.Chrome("./driver/chromedriver")
driver.get('https://sauce.foodpolis.kr/home/specialty/foodDbSearch.do?PAGE_MN_ID=SIS-030101')


page = list(range(412))
ingre_code=1
data_dict = {'name':[], 'foriegn_lang':[], 'diff':[]}
datasheet = pd.DataFrame(data_dict)

time.sleep(1)

for i in tqdm(page):
    header = {'User-Agent':'Mozila/5.0'}
    ingre = driver.page_source
    ingre_soup = BeautifulSoup(ingre, 'lxml')
    t_list = ingre_soup.select('td')
    b_list = ingre_soup.select('div.modal-body')
    

    time.sleep(0.5)
    
    for j in range(len(b_list)):
        datasheet.loc[t_list[j*3].get_text()] = [ t_list[j*3 +1].get_text(), 
                                                 t_list[j*3 +2].get_text(), 
                                                 b_list[j].get_text().split("이명")[1].split("학명")[0][1:-3]]
        ingre_code += 1
        
    if i != len(page):
        button_num = (i%5) + 4
        path = '//*[@id="content"]/div[2]/div[4]/ul/li[' + str(button_num) + ']/a'
        nextpage = driver.find_element(By.XPATH, path)
        nextpage.click()
        time.sleep(0.5)

C:\Users\BIT\AppData\Local\Temp\ipykernel_8176\3935299506.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./driver/chromedriver")


  0%|          | 0/412 [00:00<?, ?it/s]

In [154]:
datasheet

,name,foriegn_lang,diff
4933,가다랑어,"Katsuo(일명), BONITO","가다리, 에다대, 여다랭이, 강고등어(경남), 소용치, 목맨둥이, 다랭이(포항)"
4932,가다랑어포,katsuobushi,가츠오부시
4931,가리비,scallop,
4930,가리비살,Hotategai(일명),
4929,가시복,Harisenbon(일명),
...,...,...,...
5,strawberry guave열매,"Erdbeerguave, Cattley Guava, Purple Guava, pin...",
4,sugar pea,snow pea,깍지 완두 스노피
3,tea oil plant씨,tea oil camellia,올레이페라 동백
2,wild endive,wild endive,


In [134]:
datasheet.to_csv("ingredient_list.csv")

In [157]:
list(datasheet.columns)

['name', 'foriegn_lang', 'diff']